 # Part 1:  Video Upload and Frame Extraction

1. Install the Required Libraries

In [7]:
import cv2
import os

2. Directory for video uplaod and frame extraction

In [8]:

# Function to create directory for storing frames
def create_frame_dir(video_file):
    frame_dir = os.path.splitext(video_file)[0] + "_frames"
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
    return frame_dir

# Function to extract and save frames from the video
def extract_frames(video_file):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    
    # Create directory to store the frames
    frame_dir = create_frame_dir(video_file)
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # If no frame is captured, exit
        
        # Save frame as image
        frame_path = os.path.join(frame_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        frame_count += 1
        print(f"Extracted frame {frame_count}")
    
    cap.release()
    print(f"Frame extraction completed. Frames are saved in {frame_dir}")

# Call the function with your video file path
video_file = "Video2.mp4"  # Replace with your video file
extract_frames(video_file)


Frame extraction completed. Frames are saved in Video2_frames


# Part 2: Player Detection Using YOLOv5

Step 1: Install YOLOv5

In [9]:
import torch
from pathlib import Path

Step 2: Use YOLOv5 to detect players in frames.

In [10]:

# Load YOLOv5 pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Detect players in the extracted frames
def detect_players_in_frames(frame_dir):
    frame_paths = list(Path(frame_dir).glob('*.jpg'))
    
    for frame_path in frame_paths:
        # Perform detection
        results = model(frame_path)
        
        # Display results (bounding boxes around detected players)
        # results.show()  # This will show the image with detection

        # If you want to save the results, uncomment the following line
        # results.save()  # This will save images with bounding boxes to the current directory
        
        # print(f"Detected players in {frame_path}")

# Call the function with the frame directory
frame_dir = os.path.splitext(video_file)[0] + "_frames"  # Replace with your frame directory
detect_players_in_frames(frame_dir)


Using cache found in C:\Users\dhira/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-6 Python-3.12.3 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Part 3: Tracking the Player Across Frames

Step 1: Install DeepSORT

In [11]:
import numpy as np
import sklearn

print("NumPy version:", np.__version__)
print("scikit-learn version:", sklearn.__version__)



NumPy version: 1.26.4
scikit-learn version: 1.5.2


In [12]:
import sys

In [13]:
from deep_sort_realtime.deepsort_tracker import DeepSort


Step 2: Player tracking with DeepSORT

In [14]:
# Import necessary libraries
import cv2
import torch
from pathlib import Path
import numpy as np

# Initialize the YOLOv5 model (Ensure that it's already set up)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Assuming the DeepSORT object is replaced with a custom tracker or placeholder
class SimpleTracker:
    def __init__(self):
        self.next_id = 0
        self.objects = {}

    def update(self, bboxes):
        new_objects = {}
        for bbox in bboxes:
            new_objects[self.next_id] = bbox
            self.next_id += 1
        self.objects = new_objects
        return [[*bbox, obj_id] for obj_id, bbox in self.objects.items()]

# Initialize a simple tracker
tracker = SimpleTracker()

# Function to track players across frames
def track_players_in_video(video_file):
    cap = cv2.VideoCapture(video_file)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no frame is captured

        # Perform detection (similar to part 2)
        results = model(frame)
        bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
        confs = results.xyxy[0][:, 4].cpu().numpy()    # Confidence scores

        # Perform tracking using a simple tracker (replace deepsort)
        outputs = tracker.update(bboxes)

        # Draw the bounding boxes and track IDs on the frame
        for output in outputs:
            x1, y1, x2, y2, track_id = output[:5]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

        cv2.imshow("Tracking", frame)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the tracking function
video_file = "Video2.mp4"  # Replace with your video file
track_players_in_video(video_file)


Using cache found in C:\Users\dhira/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-6 Python-3.12.3 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Part 4: Pose Estimation Using OpenPose

Step 1: Install OpenPose

In [15]:
import cv2
import mediapipe as mp

Step 2: Implement Pose Estimation on Extracted Frames

In [16]:
import cv2
import torch
import mediapipe as mp

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Load YOLOv5 pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Function to perform pose estimation on a single frame
def pose_estimation_on_frame(frame):
    # Convert frame to RGB as MediaPipe works with RGB images
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform pose estimation
    results = pose.process(rgb_frame)

    # Draw keypoints on the frame
    annotated_frame = frame.copy()
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    return annotated_frame, results.pose_landmarks

# Function to detect players and perform pose estimation
def process_video(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no frame is captured

        # Perform detection using YOLOv5
        results = model(frame)
        bboxes = results.xyxy[0].cpu().numpy()  # Bounding boxes

        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Draw bounding boxes for detected players
        for bbox in bboxes:
            # Check the number of values in bbox
            if len(bbox) == 4:
                x1, y1, x2, y2 = bbox  # If only bounding box coordinates are returned
                conf = 1.0  # Set default confidence if not available
            elif len(bbox) >= 5:
                x1, y1, x2, y2, conf = bbox[:5]  # Get the first five values
            else:
                continue  # Skip if bbox has fewer than 4 values

            # Draw the bounding box
            cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)

        # Display the annotated frame with pose landmarks and bounding boxes
        cv2.imshow("Pose Estimation and Player Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function with your video file
video_file = "Video2.mp4"  # Use the video file specified earlier
process_video(video_file)


Using cache found in C:\Users\dhira/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-11-6 Python-3.12.3 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Part 5: Action Recognition

Step 1: Define Actions and Recognition Logic

In [17]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)  # First
    b = np.array(b)  # Mid
    c = np.array(c)  # End
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Function to perform pose estimation on a single frame
def pose_estimation_on_frame(frame):
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    annotated_frame = frame.copy()

    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    return annotated_frame, results.pose_landmarks

# Action recognition function
def recognize_action(landmarks):
    if landmarks is not None:
        # Define keypoints
        left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]
        
        right_shoulder = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        right_elbow = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
        right_wrist = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]

        left_hip = landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        left_knee = landmarks.landmark[mp_pose.PoseLandmark.LEFT_KNEE]
        left_ankle = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ANKLE]

        right_hip = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        right_knee = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]
        right_ankle = landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]

        # Calculate angles
        shoulder_angle_left = calculate_angle(
            (left_shoulder.x, left_shoulder.y),
            (left_elbow.x, left_elbow.y),
            (left_wrist.x, left_wrist.y)
        )

        shoulder_angle_right = calculate_angle(
            (right_shoulder.x, right_shoulder.y),
            (right_elbow.x, right_elbow.y),
            (right_wrist.x, right_wrist.y)
        )

        hip_angle_left = calculate_angle(
            (left_hip.x, left_hip.y),
            (left_knee.x, left_knee.y),
            (left_ankle.x, left_ankle.y)
        )

        hip_angle_right = calculate_angle(
            (right_hip.x, right_hip.y),
            (right_knee.x, right_knee.y),
            (right_ankle.x, right_ankle.y)
        )

        # Debug outputs for angles
        print(f"Left Shoulder Angle: {shoulder_angle_left}")
        print(f"Right Shoulder Angle: {shoulder_angle_right}")
        print(f"Left Hip Angle: {hip_angle_left}")
        print(f"Right Hip Angle: {hip_angle_right}")

        # Define action recognition based on calculated angles
        if shoulder_angle_left < 30 and shoulder_angle_right < 30:  # Shooting
            return "Action: Shooting"
        elif shoulder_angle_left > 150 and shoulder_angle_right > 150:  # Running
            return "Action: Running"
        elif 30 <= shoulder_angle_left <= 150 and 30 <= shoulder_angle_right <= 150:  # Neutral position
            return "Action: Neutral"
        elif hip_angle_left < 30 and hip_angle_right < 30:  # Jumping
            return "Action: Jumping"
        elif hip_angle_left > 160 and hip_angle_right > 160:  # Standing
            return "Action: Standing"
        elif 60 <= hip_angle_left <= 120 and 60 <= hip_angle_right <= 120:  # Bending
            return "Action: Bending"
        elif 45 <= hip_angle_left <= 90 and 45 <= hip_angle_right <= 90:  # Walking
            return "Action: Walking"
        else:
            return "Action: Unknown"  # Default case

    return "Action: Unknown"

# Process all frames for pose estimation and action recognition
def process_pose_estimation_on_video_with_action_recognition(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Debug output for landmarks
        if landmarks:
            for idx, landmark in enumerate(landmarks.landmark):
                print(f"Landmark {idx}: ({landmark.x}, {landmark.y})")

        action = recognize_action(landmarks)
        print(action)

        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the action recognition function with your video
video_file = "Video2.mp4"  # Use the video file specified earlier
process_pose_estimation_on_video_with_action_recognition(video_file)


# Part 6: Generating Feedback for Players

Step 1: Generate Feedback Based on Recognized Actions

In [18]:
# Part 6: Generating Feedback for Players

def generate_feedback(action):
    feedback = ""
    
    # Provide feedback based on the recognized action
    if action == "Action: Shooting":
        feedback = "Feedback: Focus on improving your shooting technique. Aim for accuracy."
    elif action == "Action: Running":
        feedback = "Feedback: Maintain a steady pace and good form while running."
    elif action == "Action: Neutral":
        feedback = "Feedback: Keep practicing your movements for better fluidity."
    elif action == "Action: Jumping":
        feedback = "Feedback: Focus on explosive power and proper landing technique."
    elif action == "Action: Standing":
        feedback = "Feedback: Maintain a balanced and stable stance. Avoid slouching."
    elif action == "Action: Bending":
        feedback = "Feedback: Ensure your knees are aligned with your toes and your back is straight."
    elif action == "Action: Walking":
        feedback = "Feedback: Practice smooth and controlled strides to enhance your walking efficiency."
    else:
        feedback = "Feedback: Keep practicing your actions for better performance."
    
    return feedback

# Modify the video processing function to include feedback generation
def process_pose_estimation_on_video_with_feedback(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Generate feedback
        feedback = generate_feedback(action)
        print(feedback)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the feedback generation function with your video
video_file = "Video2.mp4"  # Use the video file specified earlier
process_pose_estimation_on_video_with_feedback(video_file)


# Now Simple Implementation of getting video and anlysis them

1. Testing and Validation

In [19]:
def analyze_video(video_path):
    # Placeholder for video analysis logic
    # Replace this with actual video analysis code
    # For now, let's just return a dummy feedback
    return f"Analysis completed for {video_path}"

def test_videos(video_list):
    results = []
    for video_path in video_list:
        feedback = analyze_video(video_path)  # Call the video analysis function
        results.append((video_path, feedback))
    return results

# Sample usage with your specified video file
video_list = ['Video2.mp4']  # Use your specific video file
results = test_videos(video_list)
for video, feedback in results:
    print(f"Feedback for {video}: {feedback}")


Feedback for Video2.mp4: Analysis completed for Video2.mp4


In [20]:
action_threshold = 0.5  # Initial threshold

def set_action_threshold(new_threshold):
    global action_threshold
    action_threshold = new_threshold

# Example of updating the threshold
set_action_threshold(0.6)


2. Data Logging

In [21]:
import csv
from datetime import datetime

# Function to log feedback for each frame with timestamp
def log_feedback(video_path, action, feedback, frame_number):
    with open('feedback_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([datetime.now(), video_path, frame_number, action, feedback])

# Example of how the function would be used within your video processing loop
def process_pose_estimation_on_video_with_feedback(video_file):
    cap = cv2.VideoCapture(video_file)
    frame_number = 0  # Initialize frame counter

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Generate feedback
        feedback = generate_feedback(action)
        print(feedback)

        # Log the feedback for this frame
        log_feedback(video_file, action, feedback, frame_number)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        # Increment frame number
        frame_number += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Sample logging within video processing function
video_file = "Video2.mp4"  # Use the video file specified earlier
process_pose_estimation_on_video_with_feedback(video_file)



In [22]:
import sqlite3
from datetime import datetime

# Function to initialize the database with the appropriate table
def initialize_db():
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS performance (
            id INTEGER PRIMARY KEY,
            timestamp TEXT,
            video_path TEXT,
            frame_number INTEGER,
            action TEXT,
            feedback TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Function to insert performance data into the database
def insert_performance(video_path, frame_number, action, feedback):
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO performance (timestamp, video_path, frame_number, action, feedback) 
        VALUES (?, ?, ?, ?, ?)
    ''', (datetime.now().strftime('%Y-%m-%d %H:%M:%S'), video_path, frame_number, action, feedback))
    conn.commit()
    conn.close()

# Initialize the database
initialize_db()

# Modified video processing function to log actions and feedback frame by frame
def process_pose_estimation_on_video_with_feedback(video_file):
    cap = cv2.VideoCapture(video_file)
    frame_number = 0  # Initialize frame counter

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Generate feedback
        feedback = generate_feedback(action)
        print(feedback)

        # Insert the feedback for this frame into the database
        insert_performance(video_file, frame_number, action, feedback)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        # Increment frame number
        frame_number += 1
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the video processing function
video_file = "Video2.mp4"  # Use the video file specified earlier
process_pose_estimation_on_video_with_feedback(video_file)


3. User Interface

1)

In [23]:
# import cv2
# import mediapipe as mp
# import tkinter as tk
# from tkinter import ttk, filedialog, messagebox
# import numpy as np
# from datetime import datetime

# # Initialize Mediapipe pose detection
# mp_pose = mp.solutions.pose
# pose = mp_pose.Pose()

# # Initialize drawing utilities for Mediapipe
# mp_drawing = mp.solutions.drawing_utils

# # Calculate angle helper function
# def calculate_angle(point1, point2, point3):
#     x1, y1 = point1
#     x2, y2 = point2
#     x3, y3 = point3
#     angle = np.degrees(np.arctan2(y3 - y2, x3 - x2) - np.arctan2(y1 - y2, x1 - x2))
#     return abs(angle) if angle >= 0 else abs(angle + 360)

2)

In [24]:

# # Recognize Action (based on shoulder angle or other criteria)
# def recognize_action(landmarks):
#     if landmarks is not None:
#         left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
#         left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
#         left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

#         shoulder_angle = calculate_angle(
#             (left_shoulder.x, left_shoulder.y),
#             (left_elbow.x, left_elbow.y),
#             (left_wrist.x, left_wrist.y)
#         )

#         if shoulder_angle < 30:
#             action = "Action: Shooting"
#             feedback = "Feedback: Improve shooting accuracy."
#         elif 30 <= shoulder_angle < 45:
#             action = "Action: Dribbling"
#             feedback = "Feedback: Maintain low center of gravity."
#         elif 45 <= shoulder_angle < 75:
#             action = "Action: Standing"
#             feedback = "Maintain balance and posture."
#         elif 75 <= shoulder_angle < 105:
#             action = "Action: Jumping"
#             feedback = "Enhance explosive power and landing."
#         elif 105 <= shoulder_angle < 130:
#             action = "Action: Bending"
#             feedback = "Align knees and back properly."
#         elif 130 <= shoulder_angle < 150:
#             action = "Action: Walking"
#             feedback = "Focus on smooth strides."
#         elif 150 <= shoulder_angle <= 180:
#             action = "Action: Running"
#             feedback = "Keep steady pace and form."
#         else:
#             action = "Action: Unknown"
#             feedback = "Keep working on form."

#     else:
#         action = "Action: Unknown"
#         feedback = "No pose detected."

#     return action, feedback




3)

In [25]:
# # Calculate the distance between two points
# def calculate_distance(point1, point2):
#     return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

# # Function for performing pose estimation on each frame
# def pose_estimation_on_frame(frame):
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     result = pose.process(frame_rgb)
#     return result

# # Function to analyze video and track multiple players
# def analyze_video(video_path, treeview):
#     cap = cv2.VideoCapture(video_path)
#     frame_number = 0
#     prev_positions = {}  # To store the previous position of each player
#     actions_feedback = []  # To store actions and feedback for later display

#     if not cap.isOpened():
#         messagebox.showerror("Error", "Could not open video file.")
#         return

#     fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second of the video

#     while True:
#         ret, frame = cap.read()
#         if not ret:
#             break  # Break if no frame is returned (end of video)

#         frame_number += 1
#         result = pose_estimation_on_frame(frame)
        
#         if result.pose_landmarks:
#             # If multiple people are detected, result.pose_landmarks will be a list of NormalizedLandmarkList
#             if isinstance(result.pose_landmarks, list):
#                 for i, landmarks_person in enumerate(result.pose_landmarks):
#                     # Calculate center of mass (torso center, e.g., mid-point between shoulders and hips)
#                     left_shoulder = landmarks_person.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
#                     right_shoulder = landmarks_person.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
#                     left_hip = landmarks_person.landmark[mp_pose.PoseLandmark.LEFT_HIP]
#                     right_hip = landmarks_person.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

#                     # Calculate the torso center (average of shoulder and hip positions)
#                     torso_center_x = (left_shoulder.x + right_shoulder.x + left_hip.x + right_hip.x) / 4
#                     torso_center_y = (left_shoulder.y + right_shoulder.y + left_hip.y + right_hip.y) / 4
#                     current_position = (torso_center_x, torso_center_y)

#                     # If this is not the first frame for this player, calculate speed
#                     if i not in prev_positions:
#                         prev_positions[i] = current_position
#                         speed = 0
#                     else:
#                         prev_position = prev_positions[i]
#                         distance = calculate_distance(current_position, prev_position)
#                         time_diff = 1 / fps  # Time between frames (in seconds)
#                         speed = distance / time_diff  # Speed = distance / time

#                     prev_positions[i] = current_position  # Update the previous position

#                     # Recognize action and provide feedback
#                     action, feedback = recognize_action(landmarks_person)
#                     if action != "Action: Unknown":
#                         timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                         actions_feedback.append([timestamp, "Video", frame_number, f"Player {i+1}: {action}", feedback])

#                     # Drawing landmarks for each player detected
#                     mp_drawing.draw_landmarks(frame, landmarks_person, mp_pose.POSE_CONNECTIONS)

#                     # Display speed next to each player
#                     text_speed = f'Speed: {speed:.2f} m/s'
#                     text_action = f'Action: {action}'

#                     # Draw speed and action text near the player (torso center position)
#                     cv2.putText(frame, text_speed, 
#                                 (int(torso_center_x * frame.shape[1]), int(torso_center_y * frame.shape[0] - 20)), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
#                     cv2.putText(frame, text_action, 
#                                 (int(torso_center_x * frame.shape[1]), int(torso_center_y * frame.shape[0] + 20)), 
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

#             else:
#                 # If only one person is detected, process this as a single NormalizedLandmarkList
#                 action, feedback = recognize_action(result.pose_landmarks)
#                 if action != "Action: Unknown":
#                     timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#                     actions_feedback.append([timestamp, "Video", frame_number, action, feedback])

#                 mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

#                 # Calculate the speed of the player (similar to above)
#                 left_shoulder = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
#                 right_shoulder = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
#                 left_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
#                 right_hip = result.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

#                 torso_center_x = (left_shoulder.x + right_shoulder.x + left_hip.x + right_hip.x) / 4
#                 torso_center_y = (left_shoulder.y + right_shoulder.y + left_hip.y + right_hip.y) / 4
#                 current_position = (torso_center_x, torso_center_y)

#                 if 'prev_position' in locals():
#                     distance = calculate_distance(current_position, prev_position)
#                     time_diff = 1 / fps
#                     speed = distance / time_diff
#                 else:
#                     speed = 0

#                 prev_position = current_position

#                 # Display speed and action next to the player
#                 text_speed = f'Speed: {speed:.2f} m/s'
#                 text_action = f'Action: {action}'

#                 # Draw speed and action text near the player (torso center position)
#                 cv2.putText(frame, text_speed, 
#                             (int(torso_center_x * frame.shape[1]), int(torso_center_y * frame.shape[0] - 20)), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
#                 cv2.putText(frame, text_action, 
#                             (int(torso_center_x * frame.shape[1]), int(torso_center_y * frame.shape[0] + 20)), 
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

#         cv2.imshow("Pose Estimation", frame)

#         # Wait for 'q' key to exit
#         if cv2.waitKey(1) & 0xFF == ord('q'):
#             break

#     cap.release()
#     cv2.destroyAllWindows()

#     # After video ends, display the result in the treeview
#     for row in actions_feedback:
#         treeview.insert("", "end", values=row)


In [35]:
import cv2
import mediapipe as mp
import torch
from datetime import datetime
import numpy as np
import tkinter as tk
from tkinter import ttk, filedialog, messagebox

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Initialize Mediapipe pose detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize drawing utilities for Mediapipe
mp_drawing = mp.solutions.drawing_utils

# Load YOLOv5 model (make sure you have YOLOv5 installed and configured)
yolo_model = torch.hub.load("ultralytics/yolov5", "yolov5s", force_reload=True)  # Load pre-trained YOLOv5 small model

# Calculate angle helper function
def calculate_angle(point1, point2, point3):
    x1, y1 = point1
    x2, y2 = point2
    x3, y3 = point3
    angle = np.degrees(np.arctan2(y3 - y2, x3 - x2) - np.arctan2(y1 - y2, x1 - x2))
    return abs(angle) if angle >= 0 else abs(angle + 360)

# Recognize Action (based on shoulder angle or other criteria)
def recognize_action(landmarks):
    if landmarks is not None:
        left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

        shoulder_angle = calculate_angle(
            (left_shoulder.x, left_shoulder.y),
            (left_elbow.x, left_elbow.y),
            (left_wrist.x, left_wrist.y)
        )

        if shoulder_angle < 30:
            action = "Shooting"
            feedback = "Improve shooting accuracy."
        elif 30 <= shoulder_angle < 45:
            action = "Dribbling"
            feedback = "Maintain low center of gravity."
        elif 45 <= shoulder_angle < 75:
            action = "Standing"
            feedback = "Maintain balance and posture."
        elif 75 <= shoulder_angle < 105:
            action = "Jumping"
            feedback = "Enhance explosive power and landing."
        elif 105 <= shoulder_angle < 130:
            action = "Bending"
            feedback = "Align knees and back properly."
        elif 130 <= shoulder_angle < 150:
            action = "Walking"
            feedback = "Focus on smooth strides."
        elif 150 <= shoulder_angle <= 180:
            action = "Running"
            feedback = "Keep steady pace and form."
        else:
            action = "Unknown"
            feedback = "Keep working on form."
    else:
        action = "Unknown"
        feedback = "No pose detected."

    return action, feedback

# Calculate the distance between two points
def calculate_distance(point1, point2):
    return np.sqrt((point2[0] - point1[0]) ** 2 + (point2[1] - point1[1]) ** 2)

# Function for performing pose estimation on each frame
def pose_estimation_on_frame(frame, bbox):
    # Crop the frame to the player's bounding box
    x1, y1, x2, y2 = bbox
    cropped_frame = frame[y1:y2, x1:x2]
    cropped_frame_rgb = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2RGB)
    result = pose.process(cropped_frame_rgb)
    return result

# Function to analyze video and track multiple players
def analyze_video(video_path, treeview):
    cap = cv2.VideoCapture(video_path)
    frame_number = 0
    prev_positions = {}  # To store the previous position of each player
    actions_feedback = []  # To store actions and feedback for later display

    if not cap.isOpened():
        messagebox.showerror("Error", "Could not open video file.")
        return

    fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second of the video
    print(f"FPS of the video: {fps}")  # Check FPS to ensure it matches the expected speed

    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break if no frame is returned (end of video)

        frame_number += 1

        # Use YOLOv5 to detect players (bounding boxes)
        results = yolo_model(frame)
        bbox_list = results.xywh[0].cpu().numpy()  # Get detected bounding boxes

        for bbox in bbox_list:
            # Get bounding box coordinates
            x1, y1, w, h, conf, cls = bbox[:6]
            if conf > 0.5:  # Only consider detections with high confidence
                x1, y1, x2, y2 = int(x1 - w / 2), int(y1 - h / 2), int(x1 + w / 2), int(y1 + h / 2)

                # Perform pose estimation within the bounding box
                result = pose_estimation_on_frame(frame, (x1, y1, x2, y2))

                if result.pose_landmarks:
                    action, feedback = recognize_action(result.pose_landmarks)

                    # Only add action once per player in the frame
                    if action != "Unknown":
                        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                        actions_feedback.append([timestamp, "Video", frame_number, f"Player: {action}", feedback])

                    # Calculate player speed (based on center of the bounding box)
                    torso_center_x = (x1 + x2) / 2
                    torso_center_y = (y1 + y2) / 2
                    current_position = (torso_center_x, torso_center_y)

                    # Calculate speed in pixels per frame
                    if frame_number > 1:
                        prev_position = prev_positions.get(frame_number - 1, current_position)
                        distance = calculate_distance(current_position, prev_position)
                        time_diff = 1 / fps  # Time per frame

                        # Convert pixel distance to meters (calibration factor needed)
                        pixel_to_meter_factor = 0.05  # 1 pixel = 0.05 meters (calibrate this factor)
                        speed_in_mps = (distance * pixel_to_meter_factor) / time_diff  # Speed in meters per second

                        # Convert meters per second to kilometers per hour (1 m/s = 3.6 km/h)
                        speed_in_kmh = speed_in_mps * 3.6
                    else:
                        speed_in_kmh = 0

                    prev_positions[frame_number] = current_position

                    # Display speed and action near the player
                    text_speed = f'Speed: {speed_in_kmh:.2f} km/h'
                    text_action = f'Action: {action}'

                    # Draw speed and action text near the player (center position)
                    cv2.putText(frame, text_speed, 
                                (int(torso_center_x), int(torso_center_y - 20)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)
                    cv2.putText(frame, text_action, 
                                (int(torso_center_x), int(torso_center_y + 20)), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

        # Display the frame in a window
        cv2.imshow('Video', frame)

        # Adjust the wait time to match real-time playback speed (no delay, or minimal delay for normal playback speed)
        key = cv2.waitKey(10)  # Skip frames if you want to process faster
        if key == 27:  # Press 'Esc' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

    # Show collected actions and feedback in the Tkinter treeview
    for action_feedback in actions_feedback:
        treeview.insert("", "end", values=action_feedback)

# Function to browse video file
def browse_video_file(treeview):
    video_path = filedialog.askopenfilename(title="Select a video file", filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if video_path:
        analyze_video(video_path, treeview)

# Tkinter setup
root = tk.Tk()
root.title("Player Speed and Action Analysis")

# Create treeview widget
treeview = ttk.Treeview(root, columns=("Timestamp", "Type", "Frame Number", "Action", "Feedback"), show="headings")
treeview.heading("Timestamp", text="Timestamp")
treeview.heading("Type", text="Type")
treeview.heading("Frame Number", text="Frame Number")
treeview.heading("Action", text="Action")
treeview.heading("Feedback", text="Feedback")
treeview.pack(fill=tk.BOTH, expand=True)

# Browse button to load the video
browse_button = tk.Button(root, text="Browse Video", command=lambda: browse_video_file(treeview))
browse_button.pack(pady=10)

root.mainloop()  

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to C:\Users\dhira/.cache\torch\hub\master.zip
YOLOv5  2024-11-7 Python-3.12.3 torch-2.5.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


FPS of the video: 25.0
